### Face detection speed comparisson

In [1]:
import os
import face_recognition
import imutils
from imutils import face_utils
import pickle
import argparse
import itertools
import numpy as np
import dlib
import cv2

In [2]:
os.chdir('..')

In [3]:
! pwd

/Users/A.IVA/Documents/jupyter_notebooks/rest/coursera_and_blogs/three-eyed-raven


##### 1 Load data

In [7]:
##################################################################
# edit it such that you have the directory structure shown below #
##################################################################

# base_path = os.path.join('face-recognition-opencv', 'dataset')
base_path = os.path.join('dataset')

Expected directory structure
```
base_path
├── alan_grant
│   ├── 00000000.jpg
│   ├── 00000002.jpg
├── alan_grant
│   ├── 00000000.jpg
│   ├── 00000002.jpg
├── claire_dearing
│   ├── 00000000.jpg
│   ├── 00000002.jpg
├── ellie_sattler
│   ├── 00000000.jpg
│   ├── 00000002.jpg
├── ian_malcolm
│   ├── 00000000.jpg
│   ├── 00000002.jpg
├── john_hammond
│   ├── 00000000.jpg
│   ├── 00000002.jpg
└──  owen_grady
    ├── 00000000.jpg
    ├── 00000002.jpg
```

In [4]:
def get_images_from_dir(directory):
    
    images = os.listdir(directory)
    images = list(map(lambda x: cv2.imread(os.path.join(directory, x)), images))
    
    return images

# dirs = ['alan_grant', 'claire_dearing', 'ellie_sattler', 'ian_malcolm', 'john_hammond', 'owen_grady']
dirs = ['adrian', 'ian_malcolm', 'M S Shankar']
dirs = list(map(lambda x: os.path.join(base_path, x) , dirs))

images = list(map(lambda x: get_images_from_dir(x), dirs))
images = list(itertools.chain(*images))

len(images)

218

##### 2 Algorithms for face detection

In [5]:
#   - cv2.CascadeClassifier()
#     rectangles may be partially outside the original image
#     examples in opencv_source_code/samples/python/facedetect.py
# 
#   - dlib.get_frontal_face_detector(img, number_of_times_to_upsample)
# 
#   - face_recognition.face_locations(img=..., number_of_times_to_upsample=..., model=...)  
#     If model != 'cnn' then dlib.get_frontal_face_detector(img, number_of_times_to_upsample) is called

In [62]:
####################################
#      cv2.CascadeClassifier()     # 
####################################

detector_cascade = cv2.CascadeClassifier(os.path.join('facial-landmarks', 'haarcascade_frontalface_default.xml'))

def get_boxes_cascade(image, detector_cascade):
    
    gray = cv2.cvtColor(imutils.resize(image, width=500), cv2.COLOR_BGR2GRAY)
    
    boxes_cascade = detector_cascade.detectMultiScale(gray, scaleFactor=1.1,
                                                      minNeighbors=5, minSize=(30, 30),
                                                      flags=cv2.CASCADE_SCALE_IMAGE)
    
    if len(boxes_cascade) > 0:
        return boxes_cascade.tolist()
    return None

In [66]:
%%timeit -r2 -n1
boxes_cascade = list(map(lambda x: get_boxes_cascade(x, detector_cascade), images))

9.44 s ± 65.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [74]:
####################################
# dlib.get_frontal_face_detector() # 
####################################

detector_dlib = dlib.get_frontal_face_detector()

def get_boxes_cascade(image, detector_dlib):
    
    gray = cv2.cvtColor(imutils.resize(image, width=500), cv2.COLOR_BGR2GRAY)
    
    boxes_dlib = detector_dlib(gray, 2)
    
    return boxes_dlib

# [(left,top),(right,bottom)]
# boxes_2 = detector(gray, 2)
# boxes_2

In [75]:
%%timeit -r2 -n1
boxes_dlib = list(map(lambda x: get_boxes_cascade(x, detector_dlib), images))

56.9 s ± 595 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
